# 📘 Advanced Features: Callbacks, Robust Fitting, Auto Initial Guess⏱️ **30-40 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Advanced**---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/02_core_tutorials/advanced_features_demo.ipynb)


In [ ]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("✅ NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

## 🎯 What You'll Learn- ✓ **Monitor** optimization progress with callbacks- ✓ **Handle outliers** with robust loss functions- ✓ **Auto-generate** initial parameter guesses- ✓ **Customize** optimization algorithms- ✓ **Diagnose** convergence issues---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

from nlsq import CurveFit, auto_select_algorithm, callbacks

np.random.seed(42)

## 1. Progress Monitoring with Callbacks**Use case:** Track optimization progress in real-time

In [ ]:
from nlsq.callbacks import ProgressBar


# Define model
def exponential(x, a, b, c):
    return a * jnp.exp(-b * x) + c

# Generate data
x = np.linspace(0, 10, 100)
y = 5 * np.exp(-0.5 * x) + 2 + np.random.normal(0, 0.2, 100)

# Create callback
progress = ProgressBar(desc="Fitting exponential")

# Fit with callback
jcf = CurveFit()
popt, pcov = jcf.curve_fit(
    exponential, x, y,
    p0=[4, 0.4, 1.5],
    callback=progress
)

print(f"\nFitted parameters: {popt}")

## 2. Robust Fitting (Outlier Rejection)**Problem:** Outliers distort least-squares fits**Solution:** Use robust loss functions

In [ ]:
# Generate data with outliers
y_outliers = y.copy()
outlier_idx = np.linspace(0, len(x) - 1, num=min(5, len(x)), dtype=int)
outlier_idx = np.unique(outlier_idx)
y_outliers[outlier_idx] += np.random.uniform(3, 5, len(outlier_idx))

# Standard fit (affected by outliers)
popt_std, _ = jcf.curve_fit(exponential, x, y_outliers, p0=[4, 0.4, 1.5])

# Robust fit
popt_robust, _ = jcf.curve_fit(
    exponential, x, y_outliers,
    p0=[4, 0.4, 1.5],
    loss='huber'  # Robust loss function
)

print("Standard fit (affected by outliers):")
print(f"  Parameters: {popt_std}")
print("\nRobust fit (resistant to outliers):")
print(f"  Parameters: {popt_robust}")

# Visualize
fig = plt.figure(figsize=(10, 5))
plt.scatter(x, y_outliers, alpha=0.5, label='Data with outliers')
plt.scatter(x[outlier_idx], y_outliers[outlier_idx], c='red', s=100, label='Outliers')
plt.plot(x, exponential(x, *popt_std), 'g--', label='Standard fit', linewidth=2)
plt.plot(x, exponential(x, *popt_robust), 'r-', label='Robust fit', linewidth=2)
plt.legend()
plt.title('Robust Fitting with Outliers')
plt.tight_layout()
plt.show()

## 3. Automatic Initial Guess**Feature:** NLSQ can estimate initial parameters automatically

In [ ]:
# Fit without providing p0 (auto-guess)try:    popt_auto, _ = jcf.curve_fit(exponential, x, y)    print("Auto-generated initial guess successful!")    print(f"Fitted parameters: {popt_auto}")except Exception as e:    print(f"Auto-guess failed: {e}")    print("Tip: Provide p0 for complex models")

## 4. Algorithm Selection**NLSQ features:** Automatic algorithm selection based on problem characteristics

In [ ]:
# Auto-select algorithmrecommendations = auto_select_algorithm(exponential, x, y)print("Algorithm Recommendations:")print(f"  Algorithm: {recommendations['algorithm']}")print(f"  Tolerance: {recommendations['ftol']}")print(f"  Complexity: {recommendations.get('complexity', 'Standard')}")# Use recommended settingspopt_opt, _ = jcf.curve_fit(    exponential, x, y,    p0=[4, 0.4, 1.5],    ftol=recommendations['ftol'])print(f"\nOptimized fit: {popt_opt}")

## 🔑 Key Takeaways1. **Callbacks** enable real-time monitoring2. **Robust loss** handles outliers gracefully3. **Auto p0** simplifies fitting4. **Algorithm selection** optimizes convergence5. **Customization** gives fine control---

## 🔗 Next Steps- [Performance Optimization](performance_optimization_demo.ipynb) - Speed up fits- [Custom Algorithms](../03_advanced/custom_algorithms_advanced.ipynb) - Implement your own- [Troubleshooting Guide](../03_advanced/troubleshooting_guide.ipynb) - Diagnose issues---